### **Imports, Constants and Configurations:**

In [ ]:
import warnings
import itertools
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from utilities.utils import move_to_front

In [ ]:
NUMERICAL_COLS = [
    'budget', 'popularity', 'runtime', 'vote_average',
    'vote_count', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
    'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
    'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western',
    'language_code', 'status_code', 'year', 'month', 'day', 'actor_1_code',
    'actor_2_code', 'actor_3_code', 'actor_4_code', 'actor_5_code', 'production_company_code',
]
CREW_COLS = [
    'actor_1_code', 'actor_2_code', 'actor_3_code',
    'actor_4_code', 'actor_5_code', 'production_company_code',
]
GENRE_COLS = [
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
    'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
    'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'
]
# Numerical features that are not a genre or crew code
NON_GENRE_COLS = move_to_front(list(set(NUMERICAL_COLS) - set(GENRE_COLS) - set(CREW_COLS)), 'revenue')

In [ ]:
warnings.filterwarnings('ignore')